In [ ]:
import time
from pathlib import Path

from futurehouse_client import CrowClient, JobNames
from futurehouse_client.models import (
    AuthType,
    DockerContainerConfiguration,
    FramePath,
    JobDeploymentConfig,
    JobRequest,
    RuntimeConfig,
    Stage,
)
from ldp.agent import AgentConfig

In [ ]:
# Instantiate the client
client = CrowClient(
    stage=Stage.DEV,
    auth_type=AuthType.API_KEY,
    api_key="pOxi0pSnsJST0b7+LIp3jg.platformv01.eyJqdGkiOiJlOTI5ZGIxMy1jZDA2LTRkNjUtYTBlYi01NWYzMmQzODM1MDkiLCJzdWIiOiJOaG5kVVRkTUN2ZmkzbUZINm05S1dWTFYwU0gyIiwiaWF0IjoxNzQzMDA0NDkyLCJleHAiOjE3NDU1OTY0OTJ9.GD58gN/lDkMei3Nfw/clejJjrbL0raRKnZejzPbndfE",
)

In [ ]:
# Create a simple Job
job_data = JobRequest(
    name=JobNames.from_string("crow"),
    query="How many species of insects are there in the world?",
    n_replicates=2,
)

job_data = {
    "name": JobNames.from_string("crow"),
    "query": "What does the fox say?",
    "n_replicates": 2,
}


client.create_job(job_data)

while client.get_job().status != "success":
    time.sleep(5)
print(client.get_job())

In [ ]:
from paperqa.utils import get_citenames

bug_id = "d398d963-1e4c-4191-b9f2-c8df55376cb0"
job = client.get_job(bug_id)

print(job.answer)

get_citenames(job.answer)
# job.environment_frame['state']['state'].keys()

The answer depends on the species concept and methods used. Traditional bird taxonomies—based largely on the Biological Species Concept (BSC), which defines species on reproductive isolation—yield totals of roughly 9,000 to 11,000 recognized species. In contrast, applying the Phylogenetic Species Concept (PSC), which emphasizes diagnosable, evolutionarily independent units based on morphological and genetic differences, produces estimates of around 18,000 species or even higher.

Under traditional counts, Ernst Mayr’s 1946 assessment estimated approximately 8,600 bird species worldwide by relying on morphology and reproductive criteria, a figure that he argued was within 5–10% of the true diversity at that time (mayr1946thenumberof pages 4-7; mayr1946thenumberof pages 7-7). Similarly, Farrand and Bock’s 1980 analysis, which is based on comprehensive reviews of bird families and genera, arrived at a total near 9,021 species by compiling species counts across various families and orders 

{'Barrowclough et al. (2016',
 'barrowclough2016howmanykinds',
 'barrowclough2016howmanykinds pages 1-2',
 'barrowclough2016howmanykinds pages 2-4',
 'barrowclough2016howmanykinds pages 4-5',
 'barrowclough2016howmanykinds pages 5-8',
 'farrand1980thenumberof',
 'farrand1980thenumberof pages 13-14',
 'farrand1980thenumberof pages 5-6',
 'kiziroglu2022analysisandcomparison',
 'kiziroglu2022analysisandcomparison pages 4-6',
 'kiziroglu2022analysisandcomparison pages 9-11',
 'mayr1946thenumberof',
 'mayr1946thenumberof pages 4-7',
 'mayr1946thenumberof pages 7-7'}

In [ ]:
# Create a more complex job with runtime config

agent = AgentConfig(
    agent_type="ReActAgent",
    agent_kwargs={
        "model": "gpt-4o-mini",
        "temperature": 0.0,
    },
)
job_data = JobRequest(
    name=JobNames.from_string("dummy"),
    query="How many moons does earth have?",
    runtime_config=RuntimeConfig(agent=agent, max_steps=5),
)
client.create_job(job_data)

while client.get_job()["status"] != "success":
    time.sleep(5)
print(client.get_job())

In [ ]:
# Deploy a Crow with Frame Paths
frame_paths = [
    FramePath(path="state.pdbs", type="pdb", is_iterable=True),
    FramePath(path="state.single_pdb", type="pdb"),
]

crow = JobDeploymentConfig(
    path=Path("../../envs/dummy_env"),
    environment="dummy_env.env.DummyEnv",
    requires_aviary_internal=False,
    environment_variables={"SAMPLE_ENV_VAR": "sample_val"},
    agent="ldp.agent.SimpleAgent",
    container_config=DockerContainerConfiguration(cpu="1", memory="2Gi"),
    force=True,
    frame_paths=frame_paths,
    task_description="This is a dummy task",
)
client.create_crow(crow)

In [ ]:
while client.get_build_status()["status"] != "SUCCESS":
    time.sleep(5)
print(client.get_build_status())